## Generación base de datos NIÑO 3.4

**PROYECTO:** SISTEMA PARA EL SEGUIMIENTO DE ECOSISTEMAS VENEZOLANOS \
**AUTOR:** Javier Martinez

In [1]:
import requests
import pandas as pd
import locale

# Cambiando sistema horario
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

'es_ES.UTF-8'

### Descargando información NOAA

In [2]:
def get_detrend_nino34_ascii(url):
    """
    Funcion para tomar los datos de SST
    YR: Year de la observacion.
    MON: Mes de la observacion.
    TOTAL: Temperatura promedio observada en la region NINO3.4.
    ClimAdjust: Temperatura promedio del en la region NINO3.4 (promedio mensual)
    ANOM: Anomalias climatologicas TOTAL-ClimAdjust.
    """

    from datetime import datetime

    try:
        r =requests.get(url,allow_redirects=True)
        if r.status_code == 200:
            lista_content = r.content.decode("utf-8").split('\n')
            data_array =list(map(lambda y: list(filter(lambda x: x!='',y.split(' '))),
                            list(filter(lambda x: x!='',lista_content))
                             ))
            data_pandas = pd.DataFrame(data_array[1:],columns = data_array[0])
            data_pandas = data_pandas.rename(columns= {"YR":'year',
                                                      "MON":'month',
                                                      "TOTAL":'nino34_mean',
                                                      "ClimAdjust":'climatologica',
                                                      "ANOM":'anomalias'}).copy()
            data_pandas[['year','month']] = data_pandas[['year','month']].astype(int)
            data_pandas[['nino34_mean','climatologica','anomalias']] = data_pandas[['nino34_mean','climatologica','anomalias']].astype(float)
            data_pandas['periodo'] =  data_pandas.apply(lambda x:str(int(x.year))+'-'+str(int(x.month)),1)
            data_pandas['periodo'] =  pd.to_datetime(data_pandas['periodo'],format='%Y-%m',errors='ignore')
            data_pandas['mes'] =  data_pandas['periodo'].dt.strftime('%B')
            data_pandas = data_pandas.sort_values('periodo',ascending=True)
            data_pandas['time'] = data_pandas.periodo.apply(lambda x: datetime.toordinal(x))
            

    except:
        data_pandas = None
        
    return data_pandas.drop(['periodo'], axis=1)

In [3]:
url = 'https://origin.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/detrend.nino34.ascii.txt'

# descargando los datos
data_pandas = get_detrend_nino34_ascii(url)

data_pandas.head(10)

,year,month,nino34_mean,climatologica,anomalias,mes,time
0,1950,1,24.56,26.18,-1.62,enero,711858
1,1950,2,25.07,26.39,-1.32,febrero,711889
2,1950,3,25.88,26.95,-1.07,marzo,711917
3,1950,4,26.29,27.39,-1.11,abril,711948
4,1950,5,26.19,27.56,-1.37,mayo,711978
5,1950,6,26.47,27.21,-0.74,junio,712009
6,1950,7,26.28,26.72,-0.44,julio,712039
7,1950,8,25.88,26.30,-0.42,agosto,712070
8,1950,9,25.73,26.14,-0.41,septiembre,712101
9,1950,10,25.68,26.01,-0.32,octubre,712131


In [4]:
import json

json_data = '{ "data":' + data_pandas.to_json(orient="records") +\
                f',"url":"{url}"' + "}"

with open('./NINO34/data/detrend.nino34.ascii.json', 'w') as outfile:
    outfile.write(json_data)